In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import sys
# sys.path.insert(0, '/nas/xd/projects/transformers/src/transformers')
import os
device_mappings = {0: 1, 1: 5, 2: 6, 3: 7, 4: 2, 5: 3, 6: 0, 1: 4}
os.environ["CUDA_VISIBLE_DEVICES"] = str(device_mappings[2])

import random
import string
from collections import defaultdict, OrderedDict
from itertools import product, chain
import math
import numpy as np
import logging
import types

from pattern.en import comparative
import matplotlib.pyplot as plt
import seaborn as sns

# from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
# from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [3]:
import transformers
transformers.__version__

In file_utils.py: default_cache_path = /home/xd/.cache/huggingface/transformers


'4.18.0.dev0'

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import RandomSampler, Sampler, SequentialSampler
from enum import Enum
from typing import List, Optional, Union
# from transformers import RobertaForMaskedLM


from transformers import HfArgumentParser, Trainer, TrainingArguments, set_seed
from transformers import AutoConfig, PreTrainedModel, RobertaForMaskedLM, RobertaTokenizer
# from transformers.modeling_roberta import RobertaForProbing, RobertaDoubleHeadsModel, \
#     RobertaDoubleHeadsModel2, RobertaDoubleHeadsModel3, RobertaForSequenceClassification  # XD
# from transformers.trainer import get_mean_pred_prob  # XD
from transformers.data.data_collator import DataCollator, default_data_collator
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM, GPTJForCausalLM, XGLMForCausalLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM

logging.basicConfig(level=logging.ERROR)

In [218]:
from common_utils import *
from utils import *
models = {}
cache_dir = '/nas/xd/.cache/torch/transformers/'  # for models besides t5-3b/11b
proxies = {'http': '192.168.50.1:1081'} 

In [219]:
model_name = "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', cache_dir=cache_dir)
model = GPT2LMHeadModel.from_pretrained(model_name, proxies=proxies, cache_dir=cache_dir)
models[model_name] = model, tokenizer

In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/vocab.json
In cached_path: output_path = /nas/xd/.cache/torch/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/merges.txt
In cached_path: output_path = /nas/xd/.cache/torch/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/added_tokens.json
In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/special_tokens_map.json
In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/tokenizer_config.json
In cached_path: url_or_filename = https://huggingface.co/gpt2/resolve/main/config.json
In cached_path: output_path = /nas/xd/.cache/torch/transformers/

In [220]:
_=model.eval()

In [311]:
 import random
 input_ids = random.sample(range(200,20000),25)
 input_ids = torch.tensor(input_ids).view(1,25)
 input_ids.shape
 input_ids

torch.Size([1, 25])

tensor([[ 4346,  9512,  6004, 16602,  7755, 15780, 10244, 17558, 17792, 11566,
         14119,  8155, 13697, 11078, 15441,  1918, 13074,  1462, 19531,  7603,
           547, 11346,  9095, 19983,  5320]])

In [313]:
# input_ids = inputs.input_ids
# input_shape = input_ids.size()
# device = input_ids.device
# position_ids = torch.arange(0, input_shape[-1], dtype=torch.long, device=device)
# position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])
self = model.transformer
inputs_embeds = self.wte(input_ids) #位置编码要不要加？
# position_embeds = self.wpe(position_ids)
# hidden_states = inputs_embeds + position_embeds
# hidden_states = self.drop(hidden_states)
# hidden_states=hidden_states.to(torch.float32)
# inputs_embeds.shape
hidden_states =inputs_embeds
attn_output, _,attention ,aaa = self.h[0].attn(self.h[0].ln_1(hidden_states),output_attentions=True) #1, 25, 1280]
W_O = self.h[0].attn.c_proj.weight.split(64,0)
bias= self.h[0].attn.c_proj.bias #最终要不要加bias
aaa.shape
attention.shape

torch.Size([1, 20, 25, 64])

torch.Size([1, 20, 25, 25])

In [314]:
ans= torch.zeros(1,25,1280)
ans+=aaa[:,0,:,:].matmul(W_O[0]) #要不要加bias
# ans.shape
logits = model.lm_head(ans) #1,25,50527


In [315]:
input_ids.shape
logits.shape

torch.Size([1, 25])

torch.Size([1, 25, 50257])

In [316]:
bias = torch.zeros((25, 50257), dtype=torch.uint8)#设置mask机制

for i in range(25):
    bias[i:,input_ids[0][i]] = 1 

In [317]:
causal_mask = bias.bool().unsqueeze(0)
# causal_mask.shape
# logits.shape
logits=torch.where(causal_mask, logits, torch.tensor(-1e9))
loggailv=nn.functional.softmax(logits, dim=-1)


In [318]:
attention[0,0,:,:].shape

torch.Size([25, 25])

In [319]:
ssss=torch.max(attention[0,0,:,:],dim=-1)
ssss ##自身关注自身算不算copy head ，需要把这种情况去掉吗

torch.return_types.max(
values=tensor([1.0000, 0.9861, 0.9927, 0.9968, 0.9920, 1.0000, 0.9982, 0.9992, 0.9996,
        0.9993, 0.9988, 0.9737, 0.9984, 0.9881, 0.9823, 0.8987, 0.9988, 0.5715,
        1.0000, 0.9947, 0.8247, 0.9988, 0.9971, 0.9987, 0.9870],
       grad_fn=<MaxBackward0>),
indices=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24]))

In [304]:
input_ids.shape
tmp=input_ids[0,ssss.indices]
tmp
# loggailv[0,:,input_ids[0,ssss.indices]]
# # loggailv.sum(dim=-1).shape

loggailv[0,0,tmp[i]]

# jieguo= torch.zeros(25,1)
# for i in range(25):
#     jieguo[i][0]=loggailv[0,i,input_ids[0,ssss.indices][i]]

# jieguo


torch.Size([1, 16])

IndexError: index 16 is out of bounds for dimension 0 with size 16